In [1]:
import numpy as np
import torch
import math
from torch import nn
import matplotlib.pyplot as plt
from numba import cuda

In [2]:
# TO USE FOR COLLAB
import os
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/lib/nvidia-cuda-toolkit/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/lib/x86_64-linux-gnu/libnvvm.so"
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
!find / -iname 'libdevice'
!find / -iname 'libnvvm.so'


find: ‘/proc/65/task/65/net’: Invalid argument
find: ‘/proc/65/net’: Invalid argument
/usr/local/lib/python3.11/dist-packages/nvidia/cuda_nvcc/nvvm/libdevice
/usr/local/cuda-12.5/nvvm/libdevice
find: ‘/proc/65/task/65/net’: Invalid argument
find: ‘/proc/65/net’: Invalid argument
/usr/local/lib/python3.11/dist-packages/nvidia/cuda_nvcc/nvvm/lib64/libnvvm.so
/usr/local/cuda-12.5/nvvm/lib64/libnvvm.so


In [3]:
# Parameters
d_list = [3, 5, 10] # Dimensionality
# The number of samples is supposed to be infinite in the paper.
a_coeff = 1
alpha = 1
batch_size = 50  # Batch size for training
n_neurons = [10, 50, 1e2]
epochs = int(1e5)  # Number of training epochs
quench_epochs = int(1e4) # Number of training epochs for quench b
batch_quench = 2500 # 50^2
np.random.seed(42)
torch.random.seed()
device = 'cuda'

In [4]:
def sample_from_sphere_torch(d, n_samples, device='cuda'):
    """Generate n_samples points on the d-dimensional sphere of radius sqrt(d) using PyTorch."""
    # Sample from a Gaussian distribution
    X = torch.randn(n_samples, d, device=device)
    # Compute the norm of each sample
    norms = torch.norm(X, p=2, dim=1, keepdim=True)
    # Normalize to lie on the sphere of radius sqrt(d)
    X = (X / norms) * torch.sqrt(torch.tensor(d, device=device))
    return X

def spherical_3_spin_torch(X, a):
    """Compute the 3-spin function f(x) for all samples in X using PyTorch."""
    n_samples = X.shape[0]
    d = X.shape[1]
    f_out = torch.zeros(n_samples, device=X.device)

    # Vectorized computation of the 3-spin function
    for p in range(d):
        for q in range(d):
            for r in range(d):
                f_out += a[p, q, r] * X[:, p] * X[:, q] * X[:, r]
    # Normalize by d
    f_out /= d
    return f_out

def generate_3_spin_coefficients_torch(d, device='cuda'):
    """Generate random Gaussian coefficients a_pqr for the 3-spin model using PyTorch."""
    return torch.randn(d, d, d, device=device)

In [5]:
class SingleLayerPerceptron(nn.Module):

  def __init__(self, n_neurons, kernel, d):
    super(). __init__()
    self.y_i = nn.Linear(d, n_neurons)
    self.c_i = nn.Linear(n_neurons, 1)
    self.out = kernel

  def forward(self, x):
    x = self.out(self.y_i(x))
    x = self.c_i(x)
    return x


In [6]:
criterion = nn.MSELoss(reduction='mean').to(device)

In [7]:
# Step 4: Evaluate the Network
def evaluate_network(model, a_pqr, d):
    """Compute the loss and signed discrepancy."""
    with torch.no_grad():
      X_tensor = sample_from_sphere_torch(d, 50)
      y_tensor = spherical_3_spin_torch(X_tensor, a_pqr).reshape(-1, 1)
      y_pred = model(X_tensor)
      mse = criterion(y_pred, y_tensor)  # Mean squared error
      relative = (y_pred - y_tensor)  * (y_tensor > 0)
      signed_discrepancy = torch.mean(relative)  # Signed discrepancy for positive f(x)
      return mse, signed_discrepancy

In [8]:
import gc

In [9]:
from torch.utils.data import DataLoader, TensorDataset
from torch.cuda.amp import GradScaler, autocast
import torch
import time

def train_model(model, d, a_pqr, batch_size=50, alpha=1):
    n = model.c_i.in_features
    theta = a_coeff * n ** (-2 * alpha)
    step_size = 1e-3
    optimizer = torch.optim.SGD(model.parameters(), lr=step_size)

    # Create Dataset
    print("Create dataset:")
    t = time.time()
    X_tensor = sample_from_sphere_torch(d, epochs * batch_size)
    y_tensor = spherical_3_spin_torch(X_tensor, a_pqr).reshape(-1, 1)

    dataset = TensorDataset(X_tensor, y_tensor)
    data_loader = DataLoader(dataset, batch_size=batch_size)

    print("Time taken:", time.time() - t)
    print(f"Starting the regular training: {n}, {d}")

    # Training Loop
    t = time.time()
    scaler = GradScaler()
    for X_batch, y_batch in data_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        with autocast():
            predictions = model(X_batch)
            loss = criterion(predictions, y_batch)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

    #  Free memory after training
    # gc.collect()
    # del X_tensor, y_tensor, dataset, data_loader, X_batch, y_batch, predictions, loss
    # torch.cuda.empty_cache()  # Explicitly clear cache

    print("Time taken:", time.time() - t)
    print("Starting the quenched dataset:", n)

    #  Create Quench Dataset
    t = time.time()
    batch_quench = batch_size**2
    X_quench = sample_from_sphere_torch(d, quench_epochs * batch_quench)
    y_quench = spherical_3_spin_torch(X_quench, a_pqr).reshape(-1, 1)

    quench_dataset = TensorDataset(X_quench.reshape(quench_epochs, batch_quench, d),
                                   y_quench.reshape(quench_epochs, batch_quench, 1))
    quench_loader = DataLoader(quench_dataset, batch_size=batch_quench, shuffle=True)

    for params in optimizer.param_groups:
        params['lr'] = 1e-4

    print("Time taken:", time.time() - t)
    print("Starting the quenched training:", n)

    #  Quenched Training Loop
    t = time.time()
    for X_batch, y_batch in quench_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        with autocast():
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

    print("Time taken:", time.time() - t)

    # Free memory after training
    # gc.collect()
    # del X_quench, y_quench, quench_dataset, quench_loader, X_batch, y_batch, outputs, loss
    # torch.cuda.empty_cache()  # Clear GPU memory

    return model


Train for different d and multiple n

In [10]:
a_sample = [generate_3_spin_coefficients_torch(d_list[1]) for _  in range(3)]
result = np.zeros((len(n_neurons), len(a_sample), 2))
i = 0
for n in n_neurons:
  j = 0
  for a in a_sample:
    model = SingleLayerPerceptron(int(n), nn.Sigmoid(), d_list[1]).to(device)
    model = train_model(model, d_list[1], a, batch_size=50, alpha=1)

    # Evalute model
    result[i, j, :] = list(map(lambda x: x.detach().cpu().numpy(), evaluate_network(model, a, d_list[1])))
    gc.collect()
    torch.cuda.empty_cache()
    j += 1
  i += 1

Create dataset:
Time taken: 0.2636685371398926
Starting the regular training: 10, 5


<ipython-input-9-b295ae6dc9b7>:26: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
<ipython-input-9-b295ae6dc9b7>:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


KeyboardInterrupt: 

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import matplotlib.pyplot as plt
for i, sample in enumerate(a_sample):
    plt.plot(n_neurons, result[:, i, 0], 'o-', label=f'Sample {i+1}')
    #plt.plot(n_neurons, )
plt.xscale('log')

# Labeling and styling
plt.xlabel('Number of Neurons (log scale)')
plt.ylabel('Mean MSE')
plt.title('Mean MSE vs Number of Neurons (Log Scale)')
plt.grid(True, which="both", ls="--")  # Add grid for better readability
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
for i, sample in enumerate(a_sample):
    plt.plot(n_neurons, -result[:, i, 1], 'o-', label=f'Sample {i+1}')
plt.xscale('log')

# Labeling and styling
plt.xlabel('Number of Neurons (log scale)')
plt.ylabel('Mean MSE')
plt.title('Mean MSE vs Number of Neurons (Log Scale)')
plt.grid(True, which="both", ls="--")  # Add grid for better readability
plt.tight_layout()
plt.show()